In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import rnn_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'rnn'

PLANE = 2
CHANNEL = 11

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 15
EXECUTIONS_PER_TRIAL = 2

TOP_N = 4
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_rnn_type = hp.Choice("rnn_type", values=["lstm", "gru"])
    hp_n_neurons = hp.Choice("n_neurons", values=[16, 32, 64, 128, 256])
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=0, max_value=1, default=0)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization")

    model = bare_model_builder(hp_rnn_type, hp_n_neurons, hp_n_hidden_layers, hp_input_batch_normalization)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 lstm (LSTM)                 (None, 16)                1152      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=50),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR,
                                         project_name=PROJECT_NAME, overwrite=False)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
bayesian_tuner.results_summary(1)

Results summary
Results in ../../../data/model_selection/channel_2_11/tuner\rnn
Showing 1 best trials
Trial summary
Hyperparameters:
rnn_type: gru
n_neurons: 64
n_hidden_layers: 0
input_batch_normalization: True
Score: 8.629909992218018


In [9]:
best_model = bayesian_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 batch_normalization (BatchN  (None, 24, 1)            4         
 ormalization)                                                   
                                                                 
 gru (GRU)                   (None, 64)                12864     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 12,933
Trainable params: 12,931
Non-trainable params: 2
_________________________________________________________________


---

In [10]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 12931
========== Model 1 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 12931
========== Model 2 ==========
{'rnn_type': 'lstm', 'n_neurons': 32, 'n_hidden_layers': 1, 'input_batch_normalization': True}
Number of parameters: 12707
========== Model 3 ==========
{'rnn_type': 'gru', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 931


In [11]:
bayesian_tuner.results_summary(TOP_N)

Results summary
Results in ../../../data/model_selection/channel_2_11/tuner\rnn
Showing 4 best trials
Trial summary
Hyperparameters:
rnn_type: gru
n_neurons: 64
n_hidden_layers: 0
input_batch_normalization: True
Score: 8.629909992218018
Trial summary
Hyperparameters:
rnn_type: gru
n_neurons: 64
n_hidden_layers: 0
input_batch_normalization: True
Score: 8.644222259521484
Trial summary
Hyperparameters:
rnn_type: lstm
n_neurons: 32
n_hidden_layers: 1
input_batch_normalization: True
Score: 8.66767692565918
Trial summary
Hyperparameters:
rnn_type: gru
n_neurons: 16
n_hidden_layers: 0
input_batch_normalization: True
Score: 8.693009376525879


# Cross-validation for top 5 models

In [12]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=False)
model_scores = cross_validator()

{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 12931
Got score: 8.6151 (8.6158, 8.6145)
Got score: 8.3529 (8.3472, 8.3586)
Got score: 8.1248 (8.1328, 8.1169)
Got score: 8.4314 (8.4312, 8.4317)
Got score: 8.7513 (8.7518, 8.7507)


{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 12931
Got score: 8.6574 (8.6628, 8.6521)
Got score: 8.3729 (8.3836, 8.3622)
Got score: 8.1357 (8.1409, 8.1305)
Got score: 8.4714 (8.4740, 8.4688)
Got score: 8.7338 (8.7370, 8.7305)


{'rnn_type': 'lstm', 'n_neurons': 32, 'n_hidden_layers': 1, 'input_batch_normalization': True}
Number of parameters: 12707
Got score: 8.6809 (8.7148, 8.6471)
Got score: 8.3942 (8.3902, 8.3982)
Got score: 8.0924 (8.1019, 8.0828)
Got score: 8.6057 (8.5415, 8.6699)
Got score: 8.7913 (8.7644, 8.8183)


{'rnn_type': 'gru', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 931
Got score: 8.6376 (8.6318, 8.6434)
Got score: 8.4695 (8.4715, 8.4675)
Got score: 8.1563 (8.1846, 8.1280)
Got score: 8.5701 (8.5746, 8.5657)
Got score: 8.8209 (8.8523, 8.7895)


In [13]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.46,0.22,12931
1,8.47,0.21,12931
2,8.51,0.25,12707
3,8.53,0.22,931
